# hecho_mensajeria_accumulating

### Importación de librerías

In [23]:
import yaml
import pandas as pd
from sqlalchemy import create_engine
from datetime import datetime, timedelta, time

### Conexión a base y bodega de datos

In [24]:
with open('config.yaml') as f:
    config = yaml.safe_load(f)
    configFuente = config['fuente']
    configBodega = config['bodega']

urlFuente = f"{configFuente['driver']}://{configFuente['user']}:{configFuente['password']}@{configFuente['host']}:{configFuente['port']}/{configFuente['db']}"
urlBodega = f"{configBodega['driver']}://{configBodega['user']}:{configBodega['password']}@{configBodega['host']}:{configBodega['port']}/{configBodega['db']}"

src = create_engine(urlFuente)
etl = create_engine(urlBodega)

### Extracción y transformación de datos

In [25]:
mensajeria_estadosservicio = pd.read_sql_table('mensajeria_estadosservicio', src)

mensajeria_estadosservicio.drop(columns=["foto", "observaciones", "es_prueba", "foto_binary"], inplace=True)



def asignar_fecha_y_hora(mensajeria_estadosservicio, biblioteca, servicio_id, index, llave_fecha, llave_hora):
    dateDB = mensajeria_estadosservicio.iloc[index, 1].strftime("%Y-%m-%d")
    timeDB = mensajeria_estadosservicio.iloc[index, 2].strftime('%H:%M')
    
    biblioteca[servicio_id][llave_fecha] = dateDB
    biblioteca[servicio_id][llave_hora] = timeDB
        


def calcular_duracion(biblioteca, servicio_id, fecha_inicial, hora_inicial, fecha_final, hora_final, duracion_clave):
    fecha_a  = biblioteca[servicio_id][fecha_inicial]
    hora_a  = biblioteca[servicio_id][hora_inicial]

    fecha_b = biblioteca[servicio_id][fecha_final]
    hora_b = biblioteca[servicio_id][hora_final]

    if fecha_a  and hora_a  and fecha_b and hora_b:
    
        datetime_a   = datetime.strptime(f"{fecha_a } {hora_a }", "%Y-%m-%d %H:%M")
        datetime_b = datetime.strptime(f"{fecha_b} {hora_b}", "%Y-%m-%d %H:%M")

        duracion_minutos  = (datetime_b - datetime_a ).total_seconds() / 60
    
        biblioteca[servicio_id][duracion_clave] = duracion_minutos 



biblioteca = {}

for index, row in mensajeria_estadosservicio.iterrows():
    servicio_id = mensajeria_estadosservicio.iloc[index, 4]
    estado_id = mensajeria_estadosservicio.iloc[index, 3]

    if servicio_id not in biblioteca:
        biblioteca[servicio_id] = {
            'key_servicio': servicio_id, 
            'key_fecha_inicio': None, 
            'key_hora_inicio': None, 
            'key_fecha_asignacion': None, 
            'key_hora_asignacion': None, 
            'key_fecha_recogida': None, 
            'key_hora_recogida': None, 
            'key_fecha_entrega': None, 
            'key_hora_entrega': None, 
            'key_fecha_cierre': None, 
            'key_hora_cierre': None,
            'duracion_inicio_asignacion': None,
            'duracion_asignacion_recogida': None,
            'duracion_recogida_entrega': None,
            'duracion_entrega_cierre': None,}
    
    if estado_id == 1:
        asignar_fecha_y_hora(mensajeria_estadosservicio, biblioteca, servicio_id, index, 'key_fecha_inicio', 'key_hora_inicio')

    elif estado_id == 2:
        asignar_fecha_y_hora(mensajeria_estadosservicio, biblioteca, servicio_id, index, 'key_fecha_asignacion', 'key_hora_asignacion')

    elif estado_id == 4:
        asignar_fecha_y_hora(mensajeria_estadosservicio, biblioteca, servicio_id, index, 'key_fecha_recogida', 'key_hora_recogida')

    elif estado_id == 5:
        asignar_fecha_y_hora(mensajeria_estadosservicio, biblioteca, servicio_id, index, 'key_fecha_entrega', 'key_hora_entrega')

    elif estado_id == 6:
        asignar_fecha_y_hora(mensajeria_estadosservicio, biblioteca, servicio_id, index, 'key_fecha_cierre', 'key_hora_cierre')




for servicio_id in biblioteca:

    calcular_duracion(biblioteca, servicio_id, 'key_fecha_inicio', 'key_hora_inicio', 'key_fecha_asignacion', 'key_hora_asignacion', 'duracion_inicio_asignacion')
    calcular_duracion(biblioteca, servicio_id, 'key_fecha_asignacion', 'key_hora_asignacion', 'key_fecha_recogida', 'key_hora_recogida', 'duracion_asignacion_recogida')
    calcular_duracion(biblioteca, servicio_id, 'key_fecha_recogida', 'key_hora_recogida', 'key_fecha_entrega', 'key_hora_entrega', 'duracion_recogida_entrega')
    calcular_duracion(biblioteca, servicio_id, 'key_fecha_entrega', 'key_hora_entrega', 'key_fecha_cierre', 'key_hora_cierre', 'duracion_entrega_cierre')



hecho_mensajeria = pd.DataFrame.from_dict(biblioteca, orient='index')



hecho_mensajeria['key_fecha_inicio'] = pd.to_datetime(hecho_mensajeria['key_fecha_inicio'], errors='coerce')
hecho_mensajeria['key_fecha_asignacion'] = pd.to_datetime(hecho_mensajeria['key_fecha_asignacion'], errors='coerce')
hecho_mensajeria['key_fecha_recogida'] = pd.to_datetime(hecho_mensajeria['key_fecha_recogida'], errors='coerce')
hecho_mensajeria['key_fecha_entrega'] = pd.to_datetime(hecho_mensajeria['key_fecha_entrega'], errors='coerce')
hecho_mensajeria['key_fecha_cierre'] = pd.to_datetime(hecho_mensajeria['key_fecha_cierre'], errors='coerce')

hecho_mensajeria['key_hora_inicio'] = pd.to_datetime(hecho_mensajeria['key_hora_inicio'], format='%H:%M', errors='coerce').dt.time
hecho_mensajeria['key_hora_asignacion'] = pd.to_datetime(hecho_mensajeria['key_hora_asignacion'], format='%H:%M', errors='coerce').dt.time
hecho_mensajeria['key_hora_recogida'] = pd.to_datetime(hecho_mensajeria['key_hora_recogida'], format='%H:%M', errors='coerce').dt.time
hecho_mensajeria['key_hora_entrega'] = pd.to_datetime(hecho_mensajeria['key_hora_entrega'], format='%H:%M', errors='coerce').dt.time
hecho_mensajeria['key_hora_cierre'] = pd.to_datetime(hecho_mensajeria['key_hora_cierre'], format='%H:%M', errors='coerce').dt.time


In [26]:
hecho_mensajeria['duracion_inicio_asignacion'].fillna(round(hecho_mensajeria['duracion_inicio_asignacion'].mean(), 0), inplace=True)
hecho_mensajeria['duracion_asignacion_recogida'].fillna(round(hecho_mensajeria['duracion_asignacion_recogida'].mean(), 0), inplace=True)
hecho_mensajeria['duracion_recogida_entrega'].fillna(round(hecho_mensajeria['duracion_recogida_entrega'].mean(), 0), inplace=True)
hecho_mensajeria['duracion_entrega_cierre'].fillna(round(hecho_mensajeria['duracion_entrega_cierre'].mean(), 0), inplace=True)



def llenar_fecha_y_hora2(hecho_mensajeria, index, duracion_base, fecha_modificar, fecha_base, hora_modificar, hora_base):
    duracion  = hecho_mensajeria.loc[index, duracion_base]

    dias_d  = duracion  // 1440
    minutos_d  = duracion  % 1440

    if dias_d  == 0:
        hecho_mensajeria.loc[index, fecha_modificar] = hecho_mensajeria.loc[index, fecha_base]

        hora_base_valor  = hecho_mensajeria.loc[index, hora_base]
        
        hora_modificada  = timedelta(hours=hora_base_valor.hour, minutes=hora_base_valor.minute) + timedelta(minutes=minutos_d)

        if isinstance(hora_modificada, timedelta):
            total_segundos  = int(hora_modificada.total_seconds())
            horas = (total_segundos  // 3600) % 24 
            minutos = (total_segundos  % 3600) // 60
            segundos = total_segundos  % 60
            
            hora_modificada  = time(hour=horas, minute=minutos, second=segundos)

        hecho_mensajeria.loc[index, hora_modificar] = hora_modificada 



for index, row in hecho_mensajeria.iterrows():
    for columna, valor in row.items():
        if pd.isna(valor):
            if columna == "key_fecha_inicio":
                duracion  = hecho_mensajeria.loc[index, 'duracion_inicio_asignacion']

                dias_d  = duracion  // 1440
                minutos_d  = duracion  % 1440

                if dias_d  == 0:
                    hecho_mensajeria.loc[index, 'key_fecha_inicio'] = hecho_mensajeria.loc[index, 'key_fecha_asignacion']

                    hora_asignacion  = hecho_mensajeria.loc[index, 'key_hora_asignacion']
                    
                    hora_inicio  = (timedelta(hours=hora_asignacion .hour, minutes=hora_asignacion .minute) - timedelta(minutes=minutos_d))

                    if isinstance(hora_inicio , timedelta):
                        total_segundos  = int(hora_inicio .total_seconds())
                        horas = (total_segundos  // 3600) % 24
                        minutos = (total_segundos  % 3600) // 60
                        segundos = total_segundos  % 60
                        
                        hora_inicio  = time(hour=horas, minute=minutos, second=segundos)

                    hecho_mensajeria.loc[index, 'key_hora_inicio'] = hora_inicio 

                
            if columna == 'key_fecha_asignacion':
                llenar_fecha_y_hora2(hecho_mensajeria, index, 'duracion_inicio_asignacion', 'key_fecha_asignacion', 'key_fecha_inicio', 'key_hora_asignacion', 'key_hora_inicio')
            
            if columna == 'key_hora_asignacion':
                duracion  = hecho_mensajeria.loc[index, 'duracion_inicio_asignacion']
                minutos_d  = duracion  % 1440

                hora_inicio  = hecho_mensajeria.loc[index, 'key_hora_inicio']
                
                hora_asignacion  = (timedelta(hours=hora_inicio .hour, minutes=hora_inicio .minute) + timedelta(minutes=minutos_d))


                if isinstance(hora_asignacion , timedelta):
                    total_segundos  = int(hora_asignacion .total_seconds())
                    horas = (total_segundos  // 3600) % 24
                    minutos = (total_segundos  % 3600) // 60
                    segundos = total_segundos  % 60
                    
                    hora_asignacion  = time(hour=horas, minute=minutos, second=segundos)

                hecho_mensajeria.loc[index, 'key_hora_inicio'] = hora_inicio 

                hecho_mensajeria.loc[index, 'key_hora_asignacion'] = hora_asignacion 
                
            if columna == "key_fecha_recogida":
                llenar_fecha_y_hora2(hecho_mensajeria, index, 'duracion_asignacion_recogida', 'key_fecha_recogida', 'key_fecha_asignacion', 'key_hora_recogida', 'key_hora_asignacion') 

            if columna == "key_fecha_entrega":
                llenar_fecha_y_hora2(hecho_mensajeria, index, 'duracion_recogida_entrega', 'key_fecha_entrega', 'key_fecha_recogida', 'key_hora_entrega', 'key_hora_recogida')
            
            if columna == "key_fecha_cierre":
                llenar_fecha_y_hora2(hecho_mensajeria, index, 'duracion_entrega_cierre', 'key_fecha_cierre', 'key_fecha_entrega', 'key_hora_cierre', 'key_hora_entrega')
            
            

dim_fecha = pd.read_sql_table('dim_fecha', etl)
dim_hora = pd.read_sql_table('dim_hora', etl)


for col_fecha in ['key_fecha_inicio', 'key_fecha_asignacion', 'key_fecha_recogida', 'key_fecha_entrega', 'key_fecha_cierre']:
    hecho_mensajeria = hecho_mensajeria.merge(
        dim_fecha[['fecha', 'key_dim_fecha']], 
        left_on=col_fecha, 
        right_on='fecha', 
        how='left'
    ).drop(columns=[col_fecha]).rename(columns={'key_dim_fecha': col_fecha}).drop(columns=['fecha'])
    hecho_mensajeria[col_fecha] = pd.to_numeric(hecho_mensajeria[col_fecha], errors='coerce').astype('Int64')


for col_hora in ['key_hora_inicio', 'key_hora_asignacion', 'key_hora_recogida', 'key_hora_entrega', 'key_hora_cierre']:
    hecho_mensajeria = hecho_mensajeria.merge(
        dim_hora[['hora', 'key_dim_hora']], 
        left_on=col_hora, 
        right_on='hora', 
        how='left'
    ).drop(columns=[col_hora]).rename(columns={'key_dim_hora': col_hora}).drop(columns=['hora'])
    hecho_mensajeria[col_hora] = pd.to_numeric(hecho_mensajeria[col_hora], errors='coerce').astype('Int64')


hecho_mensajeria["key_hecho_mensajeria_acumulado"] = range(1, len(hecho_mensajeria) + 1)

hecho_mensajeria


C:\Users\alejo\AppData\Local\Temp\ipykernel_21196\2448179869.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  hecho_mensajeria['duracion_inicio_asignacion'].fillna(round(hecho_mensajeria['duracion_inicio_asignacion'].mean(), 0), inplace=True)
C:\Users\alejo\AppData\Local\Temp\ipykernel_21196\2448179869.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the i

,key_servicio,duracion_inicio_asignacion,duracion_asignacion_recogida,duracion_recogida_entrega,duracion_entrega_cierre,key_fecha_inicio,key_fecha_asignacion,key_fecha_recogida,key_fecha_entrega,key_fecha_cierre,key_hora_inicio,key_hora_asignacion,key_hora_recogida,key_hora_entrega,key_hora_cierre,key_hecho_mensajeria_acumulado
0,226,30.0,46.0,26.0,292.0,393,394,394,394,394,1438,28,74,100,392,1
1,79,4477.0,202.0,38825.0,1.0,365,368,368,395,395,821,978,1180,1125,1126,2
2,613,7.0,18.0,34.0,12.0,402,402,402,402,402,636,643,661,695,707,3
3,376,1078.0,0.0,0.0,292.0,396,397,397,397,397,1253,891,891,891,1183,4
4,7164,69.0,6.0,81.0,292.0,462,462,462,462,462,962,1031,1037,1118,1410,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28425,28464,37.0,13.0,19.0,292.0,609,609,609,609,609,769,806,819,838,1130,28426
28426,28465,10.0,52.0,49.0,292.0,609,609,609,609,609,812,822,874,923,1215,28427
28427,28466,60.0,1.0,104.0,292.0,609,609,609,609,609,844,904,905,1009,1301,28428
28428,28467,49.0,90.0,104.0,292.0,609,609,609,609,609,853,902,992,1096,1388,28429


### Carga de datos

In [27]:
hecho_mensajeria.to_sql("hecho_entrega_mensajeria_acumulada", etl, index=False, if_exists="replace")

430